---
output-file: ai.html
title: AI
---

## AI Module
This module implements the core AI-powered transcription editing system. It distinguishes between regular speech transcription and natural language edit commands, enabling a conversational editing workflow.

### Architecture

The system uses a Chat-based LLM (via lisette) that: <br>
- Maintains conversation history to understand context <br>
- Distinguishes between new content and edit instructions <br>
- Returns either "APPEND" for new text or the complete edited transcript <br>
- Tracks token usage across the entire session <br>

### Multi-Provider Support

Works with multiple LLM providers through lisette: <br>
- **OpenAI**: GPT-4o, GPT-4o-mini, GPT-3.5-turbo <br>
- **Anthropic**: Claude 3.5 Sonnet, Claude 3 Opus/Sonnet/Haiku <br>
- **Google**: Gemini 1.5 Pro/Flash <br>

In [ ]:
#| default_exp ai

In [ ]:
#| export
from lisette import *

True

## TranscriptEditor Class

The `TranscriptEditor` class manages live transcription with AI-assisted editing capabilities. It uses a Chat instance that maintains conversation history, allowing it to understand context across multiple chunks of transcription.

### Initialization

```python
editor = TranscriptEditor(model="openai/gpt-4o-mini", temperature=0.1)
```

**Parameters:** <br>
- `model`: Full model identifier in format "provider/model-name" (e.g., "openai/gpt-4o-mini") <br>

### System Prompt Strategy

The editor uses a carefully crafted system prompt that instructs the LLM to: <br>
1. Detect edit commands in natural language <br>
2. Return "APPEND" when the user is adding new content <br>
3. Return the full corrected transcript when an edit is requested <br>
4. Maintain context through conversation history <br>

This approach allows the LLM to understand references like "that", "the last part", or "the first sentence" because it has access to the full conversation history.

In [ ]:
#| export
class TranscriptEditor:
    """Manages live transcription with AI-assisted editing capabilities."""
    
    def __init__(self, model: str, temperature: float = 0.1):
        self.chat = Chat(
            model,
            sp="""You are helping with live transcription. As the user speaks, you'll receive each transcribed chunk (Each chunk being a line of text).
Your job is to:
1. Detect when the user wants to edit previous text (e.g., "change that to...", "delete the last part", "replace hamburgers with pizza")
2. When an edit is requested, return ONLY the complete corrected/edited transcript
3. When it's just new text, return ONLY the word "APPEND"
4. Keep the conversation history to understand context

Format your responses as:
- For edits: Return the full corrected transcript, each sentence on its own line
- For new text: APPEND""",
            temp=temperature
        )
        self.full_transcript = ""
        self.total_tokens = 0
    
    def process_chunk(self, chunk: str) -> dict:
        """Process a transcription chunk and determine if it's new text or an edit."""
        
        # Send the chunk with context about current transcript
        response = self.chat(chunk)
        
        result = response.choices[0].message.content.strip()
        tokens = response.usage.total_tokens if hasattr(response, "usage") else 0
        self.total_tokens += tokens
        
        # Determine if it's an append or edit
        if result.startswith("APPEND"):
            self.full_transcript += result
            action = "append"
        else:
            # It's an edit - replace full transcript
            self.full_transcript = result
            action = "edit"
        
        return {
            "transcript": self.full_transcript,
            "action": action,
            "tokens_used": tokens,
            "total_tokens": self.total_tokens
        }
    
    def get_transcript(self) -> str:
        """Get the current full transcript."""
        return self.full_transcript
    
    def reset(self):
        """Reset the transcript and chat history."""
        self.full_transcript = ""
        self.total_tokens = 0
        self.chat = Chat(
            self.chat.model,
            sp=self.chat.sp,
            temp=self.chat.temp
        )

## How It Works

### Process Flow

1. **User speaks**: "I love pizza" <br>
2. **Transcriber produces**: "I love pizza." <br>
3. **AI processes**: Detects new content → Returns "APPEND" <br>
4. **Transcript updated**: Appends the new text <br>

Then later: <br>

1. **User speaks**: "Actually, change pizza to hamburgers" <br>
2. **Transcriber produces**: "Actually, change pizza to hamburgers." <br>
3. **AI processes**: Detects edit command → Returns full edited transcript <br>
4. **Transcript updated**: Replaces entire transcript with edited version <br>

### Context Awareness

The Chat instance maintains `hist` (history) of all previous interactions. This allows it to: <br>
- Understand references to "that", "it", "the last part" <br>
- Track what has been said throughout the session <br>
- Make intelligent decisions about edit scope <br>

### Token Management

The editor tracks token usage across all API calls: <br>
- `tokens_used`: Tokens consumed by the current chunk <br>
- `total_tokens`: Cumulative tokens across the entire session <br>

This helps users monitor API costs and understand the computational overhead of AI-assisted editing.

### Error Handling

If the AI processing fails, the system gracefully degrades to simple append mode, ensuring transcription continues even if the AI service is unavailable.

In [ ]:
#| eval: false
from dotenv import load_dotenv
load_dotenv()
# Test the TranscriptEditor
editor = TranscriptEditor("openai/gpt-4o-mini")

# Simulate transcription chunks
chunks = [
    "My name is Batman.\n",
    "I love pizza.\n",
    "This transcriber is working quite well.",
    "Actually, change pizza to hamburgers.",
    "Maybe even delete that first sentence about my name."
]

for chunk in chunks:
    result = editor.process_chunk(chunk)
    print(f"\n--- Chunk: {chunk}")
    print(f"Action: {result['action']}")
    print(f"Current transcript:\n{result['transcript']}")
    print(f"Tokens used: {result['tokens_used']}")


--- Chunk: My name is Batman.

Action: append
Current transcript:

Tokens used: 155

--- Chunk: I love pizza.

Action: append
Current transcript:

Tokens used: 169

--- Chunk: This transcriber is working quite well.
Action: append
Current transcript:

Tokens used: 187

--- Chunk: Actually, change pizza to hamburgers.
Action: edit
Current transcript:
My name is Batman.  
I love hamburgers.  
This transcriber is working quite well.
Tokens used: 223

--- Chunk: Maybe even delete that first sentence about my name.
Action: edit
Current transcript:
I love hamburgers.  
This transcriber is working quite well.
Tokens used: 255
